# PsyMatrix

To load the trained PsyMatrix model used for the EMNLP24 paper, run the following code:

In [ ]:
from psymatrix.psymatrix import PsyMatrix

experiment = "emnlp24"
hidden_size = 32

psy = PsyMatrix(experiment, hidden_size)

## Train the Ranking Model

To improve prediction coverage by adding more pre-trained LLM models or datasets, follow these steps:

Update Dataset and Model Files: 

- `experiments/emnlp24/datasets.txt`: Add the new dataset ID.
- `experiments/emnlp24/models.txt`: Add the new model ID.

Update Meta-Dataset Performance Files

- `experiments/emnlp24/perf_loss.txt`: Record the negative log of the cross-entropy validation loss.
- `experiments/emnlp24/perf_loss_norm.txt`: Record normalized performance, where the lowest score is 0.0 and the highest is 1.0 (dataset wise).

Use `characterize.sh` to generate characterization for new datasets.

After updating these files, execute the code below to retrain the ranking model with the new data.

In [ ]:
import matplotlib.pyplot as plt

num_models = 25  # If you added more models, add them here as well
x_axis = list(range(1,num_models))

psy_auc, psy_topk_vec_mean, psy_topk_vec_std = psy.train_ranking_model(
    num_features=625,
    num_latent_features=2,
    metric="loss",
    k_folds=10,
    num_repeats=1,
    hidden_size=32,
)

print(f"PsyMatrix AUC (norm): {psy_auc}")

plt.plot(x, psy_topk_vec_mean, marker='o', linestyle='-', color='b', label='PsyMatrix')
plt.fill_between(x, psy_topk_vec_mean - psy_topk_vec_std, psy_topk_vec_mean + psy_topk_vec_std, color="blue", alpha=0.2)
plt.legend()
plt.show()

## Model Ranking for Optimal Selection

Evaluate the model ranking performance and compare the PsyMatrix model against baseline methods (Naive and Random) in selecting optimal models.

In [ ]:
import matplotlib.pyplot as plt

from scripts.eval_nnet import eval

num_models = 25
x_axis = list(range(1, num_models))

psy_prob, naive_prob = eval(experiment, "probability")
rnd_prob = [x/len(x_axis) for x in x_axis]

plt.xlabel("Top-K")
plt.ylabel("Optimal Selection Rate")
plt.plot(x_axis, psy_prob, marker='o', linestyle='-', color='b', label='PsyMatrix')
plt.plot(x_axis, naive_prob, marker='o', linestyle='-', color='r', label='Naive')
plt.plot(x_axis, rnd_prob, marker='', linestyle='--', color='k', label='Random')
plt.legend(loc="lower right")
plt.grid()
plt.show()


## Model Ranking for Near-Optimal Selection


In [ ]:
import matplotlib.pyplot as plt

from scripts.eval_nnet import eval, eval_perf_random

num_models = 25
x_axis = list(range(1, num_models))

psy_prob, naive_prob = eval(experiment, "performance")
rnd_prob = eval_perf_random(experiment)

plt.xlabel("Top-K")
plt.ylabel("Validation Performance of Best Model")
plt.plot(x_axis, psy_prob, marker='o', linestyle='-', color='b', label='PsyMatrix')
plt.plot(x_axis, naive_prob, marker='o', linestyle='-', color='r', label='Naive')
plt.plot(x_axis, rnd_prob, marker='', linestyle='--', color='k', label='Random')
plt.legend(loc="lower right")
plt.grid()
plt.show()